# This notebook runs a train and eval loop on models with improving consensus labels over each iteration.

In [1]:
import sys
import numpy as np
import os
sys.path.insert(0, "../")

from utils.model_training import train_models
from utils.model_training import sum_xval_folds
from utils.data_loading import get_annotator_labels
from utils.data_loading import drop_and_distribute
from utils.data_loading import get_ground_truth_data_matched
from utils.data_loading import get_and_save_improved_consensus_label
from utils.data_loading import get_and_save_consensus_labels
from cleanlab.multiannotator import get_label_quality_multiannotator # only in hui wen directory
from datetime import datetime

In [2]:
now = datetime.now() # current date and time
experiment_folder = "experiment_" + str(int(now.timestamp()))
dirName = './data/experiments/' + experiment_folder

if not os.path.exists(dirName):
    os.makedirs(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")

print(f'Experiment saved in {dirName}')

Directory  ./data/experiments/experiment_1660606626  Created 
Experiment saved in ./data/experiments/experiment_1660606626


## Dropout data values

In [3]:
# Get cifar10h dataset and dropout information from it
cifar10_infolder = './data/cifar-10h/cifar10h-raw.csv' #c10h raw data folder
max_annotations = 5

c10h_labels, c10h_true_labels, c10h_true_images = get_annotator_labels(cifar10_infolder)
c10h_labels = drop_and_distribute(c10h_labels, max_annotations)

# save c10h_results
cifar10_labels_folder = f"{dirName}/todelete_c10h_labels_range_{max_annotations}.npy"
cifar10_true_labels_folder = f"{dirName}/todelete_c10h_true_labels_range_{max_annotations}.npy"
np.save(cifar10_labels_folder, c10h_labels)
np.save(cifar10_true_labels_folder, c10h_true_labels)

# Generate and save original consensus labels
consensus_outfolder = f'{dirName}/todelete_cifar10_test_consensus_dataset_range_{max_annotations}_0.csv' #output folder for consensus labels
consensus_labels = get_and_save_consensus_labels(c10h_labels, c10h_true_labels, consensus_outfolder)

# Generate label quality of each annotator
label_quality_multiannotator = None

Total idxs dropped:  464200.0
Make sure 5.0 <= 5 and 5.0 > 0: 
Total idxs dropped:  20064.0
Make sure 5.0 <= 5 and 1.0 > 0: 


## Train models through loop

In [4]:
# Load consensus labels and train model on them
models = [
    "resnet18",
#     "swin_base_patch4_window7_224"
]

train_args = {
    "num_cv_folds": 5, 
    "verbose": 1, 
    "epochs": 1, 
    "holdout_frac": 0.2, 
    "time_limit": 60, 
    "random_state": 123
}

In [5]:
# Loop through and retrain model on better pred-probs
NUM_MODEL_RETRAINS = 10

for i in range(NUM_MODEL_RETRAINS):
    for model in models:
        # Get folders
        if i == 0:
            consensus_infolder = consensus_outfolder
        else:
            consensus_infolder = f'{dirName}/todelete_cifar10_test_consensus_dataset_range_{max_annotations}_{i-1}_{model}.csv'
        model_results_folder = f'{dirName}/todelete_cifar10_consensus_range_{max_annotations}_{i}' # + [model_type]
        consensus_outfolder = f'{dirName}/todelete_cifar10_test_consensus_dataset_range_{max_annotations}_{i}_{model}.csv'

        print(f'--INFO {i}_{model}--')
        print('Loading consensus from', consensus_infolder)
        print('Saving consensus to', consensus_outfolder)
        print('Saving model results to', model_results_folder)
        print('---------------------')
        
        # Train model
        train_models([model], consensus_infolder, model_results_folder, **train_args)
        pred_probs, labels , true_labels, images = sum_xval_folds([model], model_results_folder, **train_args)
        
        # Get label quality multiannotator
        label_quality_multiannotator = get_label_quality_multiannotator(c10h_labels,pred_probs,verbose=False) if label_quality_multiannotator is None else label_quality_multiannotator

        # Generate and save consensus labels
        _ = get_and_save_improved_consensus_label(label_quality_multiannotator, c10h_true_labels, consensus_outfolder)

modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != resnet18
root.misc.num_workers 4 != 64
root.misc.seed       42 != 634
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.train.batch_size 32 != 16
root.train.early_stop_baseline 0.0 != -inf
root.train.epochs    200 != 1
}
Saved config to /datasets/uly/multiannotator_benchmarks/0a72e00a/.trial_0/config.yaml


--INFO 0_resnet18--
Loading consensus from ./data/experiments/experiment_1660606626/todelete_cifar10_test_consensus_dataset_range_5_0.csv
Saving consensus to ./data/experiments/experiment_1660606626/todelete_cifar10_test_consensus_dataset_range_5_0_resnet18.csv
Saving model results to ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_0
---------------------
----
Running cross-validation for model: resnet18
----
Running Cross-Validation on Split: 0


Model resnet18 created, param count:                                         11181642
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 158.083506 samples/sec	accuracy=0.121250	lr=0.000100
Epoch[0] Batch [99]	Speed: 850.366471 samples/sec	accuracy=0.142500	lr=0.000100
Epoch[0] Batch [149]	Speed: 834.123396 samples/sec	accuracy=0.165417	lr=0.000100
Epoch[0] Batch [199]	Speed: 822.767267 samples/sec	accuracy=0.183438	lr=0.000100
Epoch[0] Batch [249]	Speed: 828.205522 samples/sec	accuracy=0.200000	lr=0.000100
Epoch[0] Batch [299]	Speed: 833.461843 samples/sec	accuracy=0.211042	lr=0.000100
Epoch[0] Batch [349]	Speed: 838.757213 samples/sec	accuracy=0.223036	lr=0.000100
Epoch[0] Batch [399]	Speed: 849.073456 samples/sec	accuracy=0.237187	lr=0.000100
Epoch[0] Batch [449]	Speed: 846.066964 samples/sec	accuracy=0.251806	lr=0.000100
[Epoch 0] training: accuracy=0.251806
[Epoch 0] speed: 565 samples/sec	time

Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_0_resnet18/split_0/_test_pred_probs_split_0
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_0_resnet18/split_0/_test_pred_features_split_0
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_0_resnet18/split_0/_test_labels_split_0
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_0_resnet18/split_0/_test_image_files_split_0
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_0_resnet18/split_0/_test_indices_split_0
----
Running Cross-Validation on Split: 1


modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != resnet18
root.misc.num_workers 4 != 64
root.misc.seed       42 != 320
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.train.batch_size 32 != 16
root.train.early_stop_baseline 0.0 != -inf
root.train.epochs    200 != 1
}
Saved config to /datasets/uly/multiannotator_benchmarks/6a0c418e/.trial_0/config.yaml
Model resnet18 created, param count:                                         11181642
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 183.996129 samples/sec	accuracy=0.118750	lr=0.000100
Epoch[0] Batch [99]	Speed: 858.496767 samples/sec	accuracy=0.145625	lr=0.000100
Epoch[0] Batch [149]	Speed: 850.393626 samples/sec	accuracy=0.172083	lr=0.000100
Epoch[0] Batch [199]	Speed: 834.765648 samples/sec	accuracy=0.192812	lr=0.000100
Epoch[0] Batch [249]	Speed: 840.783672 samples/sec	accuracy=0.211

...

----
Running Cross-Validation on Split: 3


Epoch[0] Batch [49]	Speed: 185.442432 samples/sec	accuracy=0.098750	lr=0.000100
Epoch[0] Batch [99]	Speed: 831.826253 samples/sec	accuracy=0.141875	lr=0.000100
Epoch[0] Batch [149]	Speed: 837.601884 samples/sec	accuracy=0.162500	lr=0.000100
Epoch[0] Batch [199]	Speed: 830.598011 samples/sec	accuracy=0.185625	lr=0.000100
Epoch[0] Batch [249]	Speed: 820.948725 samples/sec	accuracy=0.203500	lr=0.000100
Epoch[0] Batch [299]	Speed: 824.600773 samples/sec	accuracy=0.222500	lr=0.000100
Epoch[0] Batch [349]	Speed: 830.929989 samples/sec	accuracy=0.241429	lr=0.000100
Epoch[0] Batch [399]	Speed: 838.975320 samples/sec	accuracy=0.253281	lr=0.000100
Epoch[0] Batch [449]	Speed: 839.825129 samples/sec	accuracy=0.262222	lr=0.000100
[Epoch 0] training: accuracy=0.262222
[Epoch 0] speed: 596 samples/sec	time cost: 12.045029
[Epoch 0] validation: top1=0.486250 top5=0.933750
[Epoch 0] Current best top-1: 0.486250 vs previous -inf, saved to /datasets/uly/multiannotator_benchmarks/2cacd5f8/.trial_0/best_ch

Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_3/_test_pred_probs_split_3
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_3/_test_pred_features_split_3
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_3/_test_labels_split_3
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_3/_test_image_files_split_3
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_3/_test_indices_split_3
----
Running Cross-Validation on Split: 4


modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != resnet18
root.misc.num_workers 4 != 64
root.misc.seed       42 != 197
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.train.batch_size 32 != 16
root.train.early_stop_baseline 0.0 != -inf
root.train.epochs    200 != 1
}
Saved config to /datasets/uly/multiannotator_benchmarks/36844061/.trial_0/config.yaml
Model resnet18 created, param count:                                         11181642
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 185.755040 samples/sec	accuracy=0.110000	lr=0.000100
Epoch[0] Batch [99]	Speed: 826.144373 samples/sec	accuracy=0.130625	lr=0.000100
Epoch[0] Batch [149]	Speed: 837.819390 samples/sec	accuracy=0.152917	lr=0.000100
Epoch[0] Batch [199]	Speed: 815.272916 samples/sec	accuracy=0.171250	lr=0.000100
Epoch[0] Batch [249]	Speed: 821.983627 samples/sec	accuracy=0.191

Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_4/_test_pred_probs_split_4
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_4/_test_pred_features_split_4
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_4/_test_labels_split_4
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_4/_test_image_files_split_4
Saving ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_4/_test_indices_split_4
Loading ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_0/_test_pred_probs_split_0
Loading ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_0/_test_labels_split_0
Loading ./data/experiments/experiment_1660606626/todelete_cifar10_consensus_range_5_9_resnet18/split_0/_test_image_f

## Compute accuracy of model based on Accuracy (labels vs true labels) by itter after folder

In [6]:
acc_noisy_vs_true_labels = (consensus_labels['label'].values == c10h_true_labels).mean()
print(f"Accuracy ORIGINAL (consensus labels vs true labels): {acc_noisy_vs_true_labels}\n")

for model in models:
    for i in range(NUM_MODEL_RETRAINS):
        
        # Get folders
        if i == 0:
            consensus_infolder = consensus_outfolder
        else:
            consensus_infolder = f'{dirName}/todelete_cifar10_test_consensus_dataset_range_{max_annotations}_{i-1}_{model}.csv'
        model_results_folder = f'{dirName}/todelete_cifar10_consensus_range_{max_annotations}_{i}' # + [model_type]
    
        print(f'--{model} iter{i}--')
        
        out_subfolder = f"{model_results_folder}_{model}/"
        pred_probs = np.load(out_subfolder + "pred_probs.npy")
        labels = np.load(out_subfolder + "labels.npy") # remember that this is the noisy labels (s)
        images = np.load(out_subfolder + "images.npy", allow_pickle=True)
        true_labels = np.load(out_subfolder + "true_labels.npy")

        # check the accuracy
        acc_labels = (pred_probs.argmax(axis=1) == labels).mean() # noisy labels (s)
        acc_true_labels = (pred_probs.argmax(axis=1) == true_labels).mean() # true labels (y)    
        acc_noisy_vs_true_labels = (labels == true_labels).mean()

        print(f"Model: {model}")
        print(f"  Accuracy (argmax pred vs labels)                 : {acc_labels}")
        print(f"  Accuracy (argmax pred vs true labels)            : {acc_true_labels}")
        print(f"  Accuracy (consensus labels vs true labels)       : {acc_noisy_vs_true_labels}\n")

Accuracy ORIGINAL (consensus labels vs true labels): 0.9676

--resnet18 iter0--
Model: resnet18
  Accuracy (argmax pred vs labels)                 : 0.4775
  Accuracy (argmax pred vs true labels)            : 0.4817
  Accuracy (consensus labels vs true labels)       : 0.9676

--resnet18 iter1--
Model: resnet18
  Accuracy (argmax pred vs labels)                 : 0.4977
  Accuracy (argmax pred vs true labels)            : 0.5002
  Accuracy (consensus labels vs true labels)       : 0.9733

--resnet18 iter2--
Model: resnet18
  Accuracy (argmax pred vs labels)                 : 0.4829
  Accuracy (argmax pred vs true labels)            : 0.4854
  Accuracy (consensus labels vs true labels)       : 0.9733

--resnet18 iter3--
Model: resnet18
  Accuracy (argmax pred vs labels)                 : 0.4881
  Accuracy (argmax pred vs true labels)            : 0.4903
  Accuracy (consensus labels vs true labels)       : 0.9733

--resnet18 iter4--
Model: resnet18
  Accuracy (argmax pred vs labels)      